In [12]:
#write config file
import json
json_path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/"
path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/"

config = {
    "DataLoading": {"path": path, "start_date": None,"stop_date":None},
    "df_manipulation": {"col_selection": None,"keep": None,"subset":None},    
    "word_contractions": {"wordcont": True}
    
    
    
#     "Supervised":
#     "Unsupervised":
#     "SimilarityMetrics"
}

with open(json_path+'config.json', 'w') as f:
    json.dump(config,f,indent=4)

In [4]:
#read config file and call the other functions
def main(json_path):
    
    import json     
    
    with open(json_path+'config.json') as config_file:
        data = json.load(config_file)
    
    #data loading 
    path = data["DataLoading"]['path']  
    start_date = data["DataLoading"]['start_date']
    stop_date = data["DataLoading"]['stop_date']
    df = data_loading(path=path,start_date=start_date,stop_date=stop_date)
            
    #data preprocessing       
    col_selection = data["df_manipulation"]['col_selection'] 
    keep = data["df_manipulation"]['keep'] 
    subset = data["df_manipulation"]['subset']
    df = df_manipulation(df,col_selection=col_selection,keep=keep,subset=subset)
    display(df)
    
    
    method #1
    ["title","Desc","comment"]
    
    wordcont = data["df_manipulation"]["word_contractions"]
    if wordcont:
        df["title_cont"] = [word_contractions(text) for text in df["title"]]
        df["desc_cont"] = [word_contractions(text) for text in df["desc"]]
        df["comment_cont"] = [word_contractions(text) for text in df["comment"]
    
    method #2
    df (title,desc,comments) -> df(title,desc,comments) + df (title_cont,desc_cont,comments_cont)
    df (title,desc,comments) -> concat df(title,desc,comments) [df1]
                              columnbind(df,df1)
                              
    #supervised learning
    #unsupervised learning
    #similarity metrics

In [10]:
json_path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/"
main(json_path)

Files read: ['C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/(2021-08-25)1_firstSet_1.json', 'C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/(2021-08-25)3_secondSet_1.json', 'C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/(2021-10-11)3_secondSet_1.json']
Shape of df before manipulation: (2712, 15)
Shape of df after selecting columns: (2712, 15)
Number of null values in df:
 id                   0
title             1808
description          0
comments          1808
updated_date         0
hierarchy_id         0
rev                  0
tenant               0
subject              0
is_current           0
hierarchy_path       0
parent_id            0
record_type

,id,title,description,comments,updated_date,hierarchy_id,rev,tenant,subject,is_current,hierarchy_path,parent_id,record_type,row_num,file
0,1308651592,provide method to update GIO fields from git r...,Please provide a way to update GIO fields from...,"++++1562123662 fbakhda\nHi @Panceac, Cornel Eu...",2021-07-21 12:30:31.387,,8,iot_platf,support,1,/1201559436/1208431055/1308651592/,1208431055,parent,1,(2021-08-25)1_firstSet_1.json
1,1308671310,Test suite execution terminates before executi...,<p>Test suite execution finished before execut...,++++1361513318 cmoala\nsys_tsdval@GL-IAF1-V-S0...,2021-05-04 09:30:00.320,,11,iot_platf,support,1,/1201559436/1208431055/1308671310/,1208431055,parent,2,(2021-08-25)1_firstSet_1.json
2,1308673361,Cloning defects from another test cycle is not...,<p>I am trying to clone defects from another t...,++++1361514315 cmoala\nObserved that only impl...,2021-05-20 11:47:18.927,,9,iot_platf,support,1,/1201559436/1208431055/1308673361/,1208431055,parent,3,(2021-08-25)1_firstSet_1.json
3,1507656633,[Testing Only] this is enhancement only,Retest some function again.,,2020-03-13 10:16:18.703,,31,iot_platf,support,1,/1201559436/1208431055/1507656633/,1208431055,parent,4,(2021-08-25)1_firstSet_1.json
4,1507656638,[Testing Only] this is consultation only,enter the support needed at here ...,++++1661488832 prajput\nHSDES testing. Please ...,2020-06-01 09:49:55.913,,19,iot_platf,support,1,/1201559436/1208431055/1507656638/,1208431055,parent,5,(2021-08-25)1_firstSet_1.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,22012641037,,"<div><span style=""font-size: 12.18px;"">Hello,&...",,2021-03-26 13:19:20.430,,11,iot_platf,support,1,/1201559436/1208431055/22012641037/,1208431055,parent,900,(2021-10-11)3_secondSet_1.json
900,22012645565,,"<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...",,2021-05-20 13:03:09.327,,11,iot_platf,support,1,/1201559436/1208431055/22012645565/,1208431055,parent,901,(2021-10-11)3_secondSet_1.json
901,22012704243,,<div>The schedule test suite allow for the use...,,2021-04-26 10:04:12.410,,9,iot_platf,support,1,/1201559436/1208431055/22012704243/,1208431055,parent,902,(2021-10-11)3_secondSet_1.json
902,22012765885,,"<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...",,2021-06-30 00:35:58.927,,14,iot_platf,support,1,/1201559436/1208431055/22012765885/,1208431055,parent,903,(2021-10-11)3_secondSet_1.json


### Functions

In [6]:
def data_loading(path,start_date=None,stop_date=None):
    '''
    Load only files that follow agreed filename format, merge files as single dataframe.
    User can choose to 
    a) Load all json files following the agreed filename format
    b) Load only json files from specific dates by adding the start and stop dates (Note: Both start_date and
    stop_date must be used together)
    
    params:
    path [string]: path of the files, without filename
    
    start_date[None/string in YYYY-MM-DD format](optional,default is None): 
    User can choose to load files starting from start_date
    - None: no start_date is provided, all files are loaded
    - string in YYYY-MM-DD format: files starting from start_date will be loaded
    
    stop_date[None/string in YYYY-MM-DD format](optional,default is None): 
    User can choose to load files until stop_date
    - None: no stop_date is provided, all files are loaded
    - string in YYYY-MM-DD format: files until stop_date will be loaded
    '''
    from datetime import datetime,timedelta
    import pandas as pd
    import glob, os, json
    import re
    
    filenames = os.listdir(path)
    file_list=[]
    date_list = []
    df = pd.DataFrame()
    
    if start_date == None and stop_date == None :
        for file in filenames:
            # search agreed file format pattern in the filename

            pattern = r"^\(\d{4}-\d{2}-\d{1,2}\)\d+\_\D+\_\d+\.json$"

            match = re.search(pattern,file)
                
            #if match is found
            if match:
                pattern = os.path.join(path, file) #join path with file name
                file_list.append(pattern) #list of json files that follow the agreed filename
            
        print("Files read:",file_list)                   
        for file in file_list:
            with open(file) as f:
                #flatten json into pd dataframe
                json_data = pd.json_normalize(json.loads(f.read()))
                json_data = pd.DataFrame(json_data)
                #label which file each row is from 
                json_data['file'] = file.rsplit("/", 1)[-1]

            df = df.append(json_data)              
                
    else:
        #convert start and stop string to datetime
        start = datetime.strptime(start_date, "%Y-%m-%d").date()
        stop = datetime.strptime(stop_date, "%Y-%m-%d").date()
    
        #iterate from start to stop dates by day and store dates in list
        while start <= stop:
            date_list.append(start)
            start = start + timedelta(days=1)  # increase day one by one

        #convert datetime objects to string
        string_list =[d.strftime("%Y-%m-%d") for d in date_list]
#         print(string_list)
        
        for file in filenames: 
            
            # search agreed file format pattern in the filename
            for date in string_list: 
                pattern = r"\("+date+r"\)\d+\_\D+\_\d+\.json"
        
                match = re.search(pattern,file)
                
                #if match is found
                if match:
                    pattern = os.path.join(path, file) #join path with file name
                    file_list.append(pattern) #list of json files that follow the agreed filename

        print("Files read:",file_list)     
        for file in file_list:
            with open(file) as f:
                #flatten json into pd dataframe
                json_data = pd.json_normalize(json.loads(f.read()))
                json_data = pd.DataFrame(json_data)
                #label which file each row is from 
                json_data['file'] = file.rsplit("/", 1)[-1]

            df = df.append(json_data)

    return df

In [7]:
def df_manipulation(df,col_selection=None,keep=None,subset=None):
    """
    1) Column selection: Keep columns in dataframe
    2) Data impute: Impute NA rows with empty string
    3) Data duplication cleaning: Drop all duplicates or drop all duplicates except for the first/last occurrence
    
    params:
    df [dataframe]: input dataframe     
    col_selection [None/list]: - None [Default]: Keep all columns in dataframe 
                               - List: List of columns to keep in dataframe                      
                                 
    keep[None/string/False]: Choose to drop all duplicates or drop all duplicates except for the first/last occurrence
                      # - None[DEFAULT] : Drop duplicates except for the first occurrence. 
                      # - "last" : Drop duplicates except for the last occurrence. 
                      # - False : Drop all duplicates.                 
    subset[list/None]: Subset of columns for identifying duplicates, use None if no column to select
   
    """
    
    print("Shape of df before manipulation:",df.shape)

    #Column selection - user can select column(s) 
    if col_selection != None:
        df = df[col_selection]
    
    print("Shape of df after selecting columns:",df.shape)

    #---Data impute - user can impute or drop rows with NA,freq of null values before & after manipulation returned---#
    print("Number of null values in df:\n",df.isnull().sum())
  

    # impute NA values with empty string
    impute_value = ""
    df = df.fillna(impute_value)
    print("Number of null values in df after NA imputation:\n",df.isnull().sum())

    #---------Data duplication cleaning--------#
    print("Number of duplicates in the df:", df.duplicated().sum())

    #drop duplicates
    if keep==None:
        keep="first"
    df = df.drop_duplicates(subset=subset, keep=keep)

    print("Shape of df after manipulation:",df.shape)

    return df


In [8]:
def word_contractions(text):
    """
    Expand word contractions (i.e. "isn't" to "is not")
    params:
    text[string]: input string 
    """
    import contractions
    return " ".join([contractions.fix(word) for word in text.split()])   


In [ ]:
df["title_cont"] = [word_contractions(text) for text in df["title"]]
df["desc_cont"]=  [word_contractions(text) for text in df["description"]]
df["comments_cont"]=  [word_contractions(text) for text in df["comments"]]
df.head()

In [9]:
def lowercase(text):
    """
    Convert all characters to lower case
    param:
    text[string]: input string 
    """
    return text.lower() if type(text) == str else text
    

In [ ]:
# config 1: DL -> DP -> Supervised learning
# config 2: DL -> DP -> Unsupervised learning
# config 3: DL -> DP -> Similarity metrics 

In [ ]:
#read config file
# import json
# json_path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/"

# with open(json_path+'config.json') as config_file:
#     data = json.load(config_file)

# path = data["DataLoading"]['path']
# start_date = data["DataLoading"]['start_date']
# stop_date = data["DataLoading"]['stop_date']
# print(path)
# print(start_date)
# print(stop_date)
